Algorithm for extraction of approximate vortex boundaries from $ \mathrm{\overline{TRA}_{t_0}^{t_N}} $.

1. Reconstruct $ \mathrm{\overline{TRA}_{t_0}^{t_N}} $ field at time t using linear radial basis function interpolation
2. Find local maxima of $ \mathrm{\overline{TRA}_{t_0}^{t_N}} $ which are above a threshold $ \mathrm{\overline{TRA}_{loc,max}} $
3. Compute for each closed level set surrounding a local maximum of $ \mathrm{\overline{TRA}_{t_0}^{t_N}} $ the median of $ | \nabla \mathrm{\overline{TRA}_{t_0}^{t_N}} | $. Find closed level set around each local maximum with the maximum median of $ | \nabla \mathrm{\overline{TRA}_{t_0}^{t_N}} | $ which contains at least N-drifters.
4. Take the convex hull of all selected closed curves
5. If two or more closed curves intersect, then take the union of these curves.
6. Take the convex hull of the resulting closed curves

In [ ]:
# Import shapely objects
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union

# Import numpy
import numpy as np

# Import scipy
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy.interpolate import RectBivariateSpline as RBS

# Import function to display progress bar
from tqdm.notebook import tqdm

# Import skimage
from skimage.feature import peak_local_max
from skimage import measure

In [2]:
# Import shapely objects
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union

# Import numpy
import numpy as np

# Import scipy
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy.interpolate import RectBivariateSpline as RBS

# Import function to display progress bar
from tqdm.notebook import tqdm

# Import skimage
from skimage.feature import peak_local_max
from skimage import measure


def __extract_vortex__(X, Y, TRA, x_pos, y_pos, threshold_TRA_loc_max = 0.7, N_drifters = 1):
    '''
    Extract vortex boundaries from TRA field subject to a set of contraints (threshold_TRA_loc_max, N_drifters)
    
    Parameters:
        X, Y:                  array(Ny, Nx), gridded X,Y domain 
        TRA:                   array(Ny, Nx), TRA-field
        x_pos, y_pos:          float, x/y coordinates of local maximum
        threshold_TRA_loc_max: float, local threshold on LAVD to find local maxima
        N_drifters:            float, minimum required number of trajectories inside an eddy
        
    Returns:
        coord_boundary_x, coord_boundary_y: list, list containing the x/y coordinates of the vortex boundaries
    '''
    
    # interpolant object for TRA-field
    interpolant_TRA = RBS(Y[:,0], X[0,:], TRA) # RectBivariateSpline-object, Bivariate spline approximation over a rectangular mesh.
    
    # spacing of auxiliary grid for derivatives
    rho_x = 0.1*(X[0,1]-X[0,0]) # float, spacing of auxiliary grid in x-direction
    rho_y = 0.1*(Y[1,0]-Y[0,0]) # float, spacing of auxiliary grid in y-direction
    
    # resolution of scalar field
    res = (np.max(TRA)-np.min(TRA))/100 # float, resolution of TRA field
    resolution_x = X[0,1]-X[0,0] # float, resolution of x
    resolution_y = Y[1,0]-Y[0,0] # float, resolution of y
    
    # find local maxima in the TRA-field above a given threshold
    xy = peak_local_max(TRA, threshold_abs = threshold_TRA_loc_max) # array(N, 2), (row, column) coordinates of N-peaks.
    
    # Define list which stores the vertices of the vortex boundary
    vertex_x_hyper_lst, vertex_y_hyper_lst = [], []
    
    # Define list which stores location and TRA-value of local maxima
    x_max_lst, y_max_lst, TRA_local_maxima = [], [], []

    # associate local maxima to corresponding lists
    for idx_maxima in range(xy.shape[0]):
        x_max_lst.append(X[xy[idx_maxima, 0], xy[idx_maxima, 1]])
        y_max_lst.append(Y[xy[idx_maxima, 0], xy[idx_maxima, 1]])
        TRA_local_maxima.append(TRA[xy[idx_maxima, 0], xy[idx_maxima, 1]])
    
    convex_contour_x, convex_contour_y = [], []
    
    # iterate over all local maxima
    for idx_maxima in tqdm(range(xy.shape[0])): 
        
        x_max = x_max_lst[idx_maxima]
        y_max = y_max_lst[idx_maxima]
        
        # Define candidate level set values which could be vortex boundaries
        levels = np.arange(0, TRA_local_maxima[idx_maxima]+res, res)
        
        vertex_x_lst, vertex_y_lst, grad_TRA_lst = [], [], []
        
        # Iterate through all the level sets and check if vortex criteria are satisfied
        for value in levels:

            # find contour to corresponding level set
            contours = measure.find_contours(TRA, value)
            
            # iterate over all contours with the given level set
            for vertices in contours:
                
                # long, lat coordinates of contour
                x_path = np.min(X)+vertices[:, 1]*resolution_x
                y_path = np.min(Y)+vertices[:, 0]*resolution_y
                
                if len(x_path) > 4:
                
                    # transfrom vertices to points
                    points = np.array([x_path, y_path]).T
                    
                    # check if contour is closed
                    if points[0, 0] == points[-1, 0] and points[0, 1] == points[-1, 1]:
                        
                        # compute Polygon object out of points
                        Poly = Polygon(points)
                
                         # Check if boundary contains a local maximum
                        check_bool_local_max = Poly.contains(Point(x_max, y_max))
                        
                        if check_bool_local_max:
                            
                            # check if closed curve contains drifters
                            check_bool_in = []
                            for idx_x in range(len(x_pos)):
                                check_bool_in.append(Poly.contains(Point(x_pos[idx_x], y_pos[idx_x])))
                                                            
                                # if number of drifters inside the closed curve is greater than N_drifters
                                # then vortex satisfies all the criteria
                                check_bool = (np.sum(check_bool_in) >= N_drifters)
                                
                                if check_bool:
                                    break
                                    
                            if np.sum(check_bool_in) >= N_drifters:
                                
                                vertex_x = points[:, 0]
                                vertex_y = points[:, 1]
                                vertex_x_lst.append(vertex_x)
                                vertex_y_lst.append(vertex_y) 
                                
                                x0 = vertex_x.ravel()
                                y0 = vertex_y.ravel()
                                
                                xR = x0+rho_x
                                xL = x0-rho_x
                                
                                yU = y0+rho_y
                                yD = y0-rho_y
                                
                                dTRAdx = (interpolant_TRA(y0, xR, grid = False)-interpolant_TRA(y0, xL, grid = False))/(2*rho_x)
                                dTRAdy = (interpolant_TRA(yU, x0, grid = False)-interpolant_TRA(yD, x0, grid = False))/(2*rho_y)
                                
                                grad_TRA_avg = np.median(np.sqrt(dTRAdx**2+dTRAdy**2))
                                
                                grad_TRA_lst.append(grad_TRA_avg)
        
        if len(grad_TRA_lst) > 0:
            
            # sort closed curves from maximum to minimum area 
            grad_TRA_sorted, vertex_x_sorted = zip(*sorted(zip(grad_TRA_lst, vertex_x_lst), reverse = True))
            grad_TRA_sorted, vertex_y_sorted = zip(*sorted(zip(grad_TRA_lst, vertex_y_lst), reverse = True))
        
            points = np.array([vertex_x_sorted[0], vertex_y_sorted[0]]).transpose()
            
            if points.shape[0] > 4:
        
                hull = ConvexHull(points, incremental=True)
                
                convex_contour_x.append(np.append(points[hull.vertices, 0], points[hull.vertices[0], 0]))
                convex_contour_y.append(np.append(points[hull.vertices, 1], points[hull.vertices[0], 1]))
    
    # if two convex curves intersect --> unite them and generate a new convex curve 
    # given by the convex hull of the two intersecting curves
    
    Poly_objects = []
    
    for i in range(len(convex_contour_x)):
        Poly_objects.append(Polygon(np.array([convex_contour_x[i].ravel(), convex_contour_y[i].ravel()]).T))
    
    coord_boundary_x, coord_boundary_y = [], []
    
    # Take union of polygons
    Multi_poly = unary_union(Poly_objects)
    
    if np.str(type(Multi_poly)) == "<class 'shapely.geometry.polygon.Polygon'>":
        
        # get array from Multi_poly object
        x = np.asarray(Multi_poly.exterior.coords)[:,0]
        y = np.asarray(Multi_poly.exterior.coords)[:,1]
        
        # define Point-object
        points = np.array([x, y]).transpose()
    
        # take convex hull of points
        hull = ConvexHull(points)
        
        coord_boundary_x.append(np.append(points[hull.vertices, 0], points[hull.vertices[0], 0]))
        coord_boundary_y.append(np.append(points[hull.vertices, 1], points[hull.vertices[0], 1]))
        
        return coord_boundary_x, coord_boundary_y
    
    for poly in Multi_poly:
        # convex hull of  poly object
        x = np.asarray(poly.exterior.coords)[:,0]
        y = np.asarray(poly.exterior.coords)[:,1]
        
        points = np.array([x, y]).transpose()
        
        hull = ConvexHull(points)
        
        coord_boundary_x.append(np.append(points[hull.vertices, 0], points[hull.vertices[0], 0]))
        coord_boundary_y.append(np.append(points[hull.vertices, 1], points[hull.vertices[0], 1]))
                        
    return coord_boundary_x, coord_boundary_y